In [32]:
# question_templates={
#     'missing_step' : 'Qual è lo step mancante nella seguente lista di step per la guida: {title}. Steps: {list_of_steps}',

#     'next_step' : 'Sapendo che il titolo della guida è {step}.Qual è il prossimo step in questa lista: {list_of_steps}',

#     'open_question' : 'Write the steps for this guide: {title}',

#     'rearrange' : 'Rearrange the steps for {title}: {list_of_steps}'
# }

# Questions
- remove a single step but print in the question all steps: step1 step2 ... step4 -> correct answer: step3
- predict the next step (minimum 4 steps before): step1 2 3 4 ... -> correct answer: step5
- Sequence Rearrangement. Provide the steps in a jumbled order and ask the user to arrange them in the correct   sequence.
Example:
Question: Rearrange the steps for “How to Tie a Shoelace”: Step 3, Step 1, Step 2, Step 4.
Correct Answer: Step 1, Step 2, Step 3, Step 4.
- open question write the steps for this guide
- difficulty: take the option step between the same guide (maximum difficulty) or from a different guide (easy)

In [33]:
import requests
from bs4 import BeautifulSoup,NavigableString
import random
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
import re
from tqdm.asyncio import tqdm
import asyncio
import aiohttp
from ollama import AsyncClient
import ollama

In [34]:
url='https://www.wikihow.com/Category:Gardening'
def get_all_pages(url,releted=True):
    base_url='https://www.wikihow.com'
    response = requests.get(url)
    pages=set()
    categories=set()
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        cat_sections=soup.find_all('div',class_='cat_section')
        for cat_section in cat_sections:
            if cat_section.get('id')=='cat_featured':
                #print('Feautured')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_all':
                #print('ALL')
                articles=cat_section.find_all('div',class_='responsive_thumb')
                for article in articles:
                    url_a=article.find('a').get('href')
                    pages.add(url_a)
            if cat_section.get('id')=='cat_sub_categories' and releted:
                #print('SUBCAT')
                uls=cat_section.find_all('ul')
                for ul in uls:
                    for li in ul.find_all('li'):
                        #print(li)
                        for a in li.find_all('a',class_='cat_link'):
                            categories.add(base_url+a.get('href'))
        return pages,categories

In [35]:
urls,categories=get_all_pages(url,True)


In [36]:
urls2= set()
for category in categories:
    u,c = get_all_pages(category,False)
    urls2.update(u)
    if c:
        print(c)

In [37]:
urls

{'https://www.wikihow.com/Adjust-Hunter-Sprinklers',
 'https://www.wikihow.com/Apply-Liquid-Fertilizer',
 'https://www.wikihow.com/Apply-Mulch',
 'https://www.wikihow.com/Arrange-the-Inside-of-a-Greenhouse',
 'https://www.wikihow.com/Build-a-Lean-to-Shed',
 'https://www.wikihow.com/Build-a-Pergola',
 'https://www.wikihow.com/Clean-Garden-Stones',
 'https://www.wikihow.com/Clean-a-Slate-Hearth',
 'https://www.wikihow.com/Cut-Back-Irises-in-the-Fall',
 'https://www.wikihow.com/Cut-Back-Penstemon',
 'https://www.wikihow.com/Deal-with-Exposed-Tree-Roots',
 'https://www.wikihow.com/Determine-How-Much-Water-Plants-Need',
 'https://www.wikihow.com/Garden',
 'https://www.wikihow.com/Get-Rid-of-Morning-Glories',
 'https://www.wikihow.com/Grow-Bloodworms',
 'https://www.wikihow.com/Grow-Daikon',
 'https://www.wikihow.com/Grow-Pumpkins-Indoors',
 'https://www.wikihow.com/Grow-Sweet-Potato-Vine-Houseplant',
 'https://www.wikihow.com/Grow-Taro',
 'https://www.wikihow.com/Grow-Zucchini-in-Pots',
 'h

In [38]:
#url = 'https://www.wikihow.com/Prune-a-Tree'
def get_page(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        return {'title': url.split('/')[-1], 'soup':soup}
    else:
        print("Failed to retrieve the page.")

In [39]:
urls.update(urls2)

In [40]:
soups=[]
for url in tqdm(list(urls)):
    soup = get_page(url)
    soups.append(soup)

100%|██████████| 996/996 [03:07<00:00,  5.32it/s]  


In [41]:
class Page:
    def __init__(self, data):
        self.title = data['title']
        self.sections = [Section(section) for section in data['sections']]

    def display_sections(self):
        return [section.display() for section in self.sections]
    
    def __str__(self):
        output = f'Page Title: {self.title}\nURL: https://www.wikihow.com/{self.title.replace(" ","-")}\nSections:\n'
        for section in self.sections:
            output += str(section) + '\n'
        return output

class Section:
    def __init__(self, section_data):
        self.title = section_data['section_title']
        self.type = section_data['section_type']
        self.steps = []
        for sub_i ,subsection in enumerate(section_data['subsections']):
            #self.subsection.append([])
            for i,step_data in enumerate(subsection,1):
                self.steps.append(Step(step_data, i))

    def display(self):
        return f'({self.type}) {self.title} with {len(self.steps)} steps.'
    
    def __str__(self):
        output = f'- ({self.type}) {self.title}  with {len(self.steps)} steps.\n'
        for step in self.steps:
            output += str(step) + '\n'
        return output

class Step:
    def __init__(self, step_data, index):
        self.index = index
        self.image = step_data['image']
        self.title = step_data.get('title', '')  # Use get() to provide a default value
        self.text = step_data.get('text', '')   # Use get() to provide a default value

    def display(self):
        return f'       Step {self.index}: {self.title}\n            {self.text[:50]}...'
    
    def __str__(self):
        return f'   -- Step {self.index}: {self.title} - {self.text[:50]}...'

In [42]:
def adjust_page(page):
    steps=[]
    for num,section in enumerate(page.sections,1):
        new_step=Step({'title' : section.title,'text' : section.steps[0].title+' '+section.steps[0].text,'image':section.steps[0].image},index=num)
        #print(new_step)
        steps.append(new_step)
    page.sections[0].steps = steps
    page.sections[0].type='No type'
    page.sections[0].title='Steps'
    #page.sections[0].steps = steps
    page.sections = [page.sections[0]]

In [43]:
def process_soup(title,soup):
    title=title
    if title in ['Prune-Nandinas','Kill-Poison-Oak','What-Plant-Am-I','Meaning-of-the-Carnation','Grow-Mint-from-Cuttings']:
        return None
    #sections = soup.find_all('div',class_=lambda x: x and x.startswith('section steps'))
    sections = soup.find_all('div', class_=lambda x: x and (x in ['section steps steps_first sticky hide_step_numbers','section steps sticky hide_step_numbers','section steps before_steps_section sticky', 'section steps steps_first sticky', 'section steps sticky']))# == 'section steps steps_first sticky' or x == 'section steps sticky'))
    
    sections_dir=[]
    if len(sections)==0:
        print('No Section',title)
        return None
    for section in sections:
        h3_tag = section.find_all('h3')
        type = section.find_all('div',class_='altblock')
        section_type = 'No type'
        if type:
            section_type = type[0].text.strip()
        #print(type[0].text)
        if h3_tag:
            section_title = h3_tag[0].text.strip()
            #print(title,' -> ',section_title)
            #sections_dir.append(section_title)
        else:
            h2_tag = section.find_all('h2')
            if h2_tag:
                section_title = h2_tag[0].text.strip()
                #sections_dir.append(section_title)
        
        ols=section.find_all('ol',class_=lambda x: x and x.startswith('steps'))
        if len(ols)>1:
            print('Attention more than one ol',title)

            #return
        #part=0
        page=[]
        
        divs=[]
        subsections=[]
        for ol in ols:
            #section=[]
            #part+=1
            step_num=0
            subsection=[]
            for li in ol.find_all('li', id=lambda x: x and x.startswith('step-id')):
                step={}
                step_num+=1
                img_tag = li.find('img')
                if img_tag and img_tag.get('data-src'):
                    step_image = img_tag['data-src']
                    step['image'] = step_image
                else:
                    # Handle missing image or data-src attribute
                    step['image'] = None
                step_div=li.find('div', class_='step')
                for element in step_div.children:
                    if isinstance(element, NavigableString) and element.strip():
                        #non_tag_text += element.strip() + " "
                        text = step.get('text','')
                        text += element.strip() + " "
                        step['text'] = text
                    # Check if element has the class 'whb'
                    elif element.name and 'whb' in element.get("class", []):
                        step_title = step.get('title','')
                        step_title += element.get_text(strip=True)
                        step['title'] = step_title
                    elif element.name == 'a' and 'external' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                    elif element.name == 'span' and 'frac' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        fraction = element.text.replace('⁄','/')
                        text += fraction + " "
                        step['text'] = text
                    elif element.name == 'span' and 'mwe-math-element' in element.get("class", []):
                        #non_tag_text += element.text + " "
                        text = step.get('text','')
                        fraction = element.text.replace('⁄','/')
                        text += fraction + " "
                        step['text'] = text
                    elif element.name == 'a' and (not element.get("class") or 'intra-article' in element.get("class")):
                        #non_tag_text += element.text + " "
                        found=False
                        for child in element.children:
                            if child.name and 'whb' in child.get("class", []):
                                step_title = step.get('title','')
                                step_title += child.get_text(strip=True)
                                step['title'] = step_title 
                                found=True
                        if not found:
                            text = step.get('text','')
                            text += element.text + " "
                            step['text'] = text
                    elif element.name == 'i' and not element.get("class"):
                        #non_tag_text += element.text + " " 
                        text = step.get('text','')
                        text += element.text + " "
                        step['text'] = text
                    elif element.name == 'b' and not element.get("class"):
                        #non_tag_text += element.text + " "
                        found=False
                        for child in element.children:
                            if child.name and 'whb' in child.get("class", []):
                                step_title = step.get('title','')
                                step_title += child.get_text(strip=True)
                                step['title'] = step_title 
                                found=True
                        if not found:
                            text = step.get('text','')
                            text += element.text + " "
                            step['text'] = text
                    elif element.name == 'div':
                        #print(f'Div Found in part {section_type} step: {step_num} | Article: {title}')
                        divs.append(element)
                    elif element.name == 'ul':
                        ul_elem=[]
                        for li in element.find_all('li'):
                            plain_text = ''.join(text for text in li.contents if isinstance(text, str)).strip()
                            ul_elem.append(plain_text)
                        step_list = ''
                        for elem in ul_elem:
                            step_list += '\n- ' + elem
                        text = step.get('text','')
                        text += step_list + " "
                        step['text'] = text
                        #step['ul_lists']= step_list
                    # Check if element is another tag
                    elif element.name in ['script', 'sup','br','p','span','table']: #table could be processed
                        pass
                    elif element.name:
                        print(f"Other tag ({element.name}) found in part {section_type} step: {step_num} | Article: {title} |: {element}")
                    else:
                        pass
                if not step.get('title') or len(step['title'])<5:
                    print('Problem: ',title,'in the step',step.get('title'),f'found in part {section_type} step: {step_num}')
                subsection.append(step) 
            subsections.append(subsection)
        sections_dir.append({'section_title' : section_title,'section_type' : section_type, 'subsections':subsections})
    return {'title':title,'sections' : sections_dir}#, divs

In [44]:
pages={}
for soup in tqdm(soups):
    processed_soup=process_soup(soup['title'],soup['soup'])
    if processed_soup:
        page=Page(processed_soup)
        if  page.sections[0].type.startswith('1'):
            adjust_page(page)
    #Page(pages['Plant-an-Avocado-Tree'])
        pages[soup['title']] = page

  7%|▋         | 65/996 [00:00<00:05, 155.90it/s]

Attention more than one ol Grow-Avocados
Attention more than one ol Sharpen-Lawn-Mower-Blades


 43%|████▎     | 431/996 [00:02<00:04, 128.05it/s]

Attention more than one ol Graft-a-Fruit-Tree


 66%|██████▌   | 655/996 [00:04<00:01, 176.65it/s]

...

 81%|████████  | 802/996 [00:05<00:01, 164.60it/s]

Attention more than one ol Grow-Edible-Mushrooms


 90%|████████▉ | 893/996 [00:05<00:00, 172.36it/s]

Attention more than one ol Plant-an-Avocado-Tree


100%|██████████| 996/996 [00:06<00:00, 161.11it/s]


In [45]:
def all_one(page):    
    for section in page.sections:
        if len(section.steps)>1:
            return False
    return True

In [46]:
c=0
types=set()
for page in pages:
    #if pages[page].sections[0].type == 'No section':
    types.add( pages[page].sections[0].type)
    #if  not pages[page].sections[0].type:#.startswith('No'):
    if  pages[page].sections[0].type.startswith('P'):
        print(pages[page])
        c+=1
        #adjust_page(pages[page])
        #print(pages[page])
print(c,'/',len(pages))

Page Title: Harvest-Cauliflower
URL: https://www.wikihow.com/Harvest-Cauliflower
Sections:
- (Part 1) Harvesting Your Cauliflower Head  with 5 steps.
   -- Step 1: Harvest your cauliflower in the fall if you planted in the spring. - Cauliflower is a cold-weather crop, but you may be...
   -- Step 2: Harvest your cauliflower in the winter if you planted it in the fall. - Since cauliflower is a cold-weather crop, you shou...
   -- Step 3: Harvest when the head grows to 6–8 in (15–20 cm) and is white and firm. - Healthy cauliflower grows 1 large head in the cent...
   -- Step 4: Chop your cauliflower head at the base of the plant using a sharp knife. - To harvest your cauliflower, make a cut at a 45-de...
   -- Step 5: Remove any remaining leaves to either dispose of or cook with. - The head of the cauliflower plant grows directly i...

- (Part 2) Harvesting Florets and Leaves  with 3 steps.
   -- Step 1: Continue giving your plants water every week. - Your cauliflower plant will continue

In [47]:
types

{'', 'Method 1', 'No type', 'Part 1'}

In [48]:
page_t=list(pages.keys() )[random.randint(0,len(pages))]
print(pages[page_t])#['sections'][0]['steps'][0]

Page Title: Get-Rid-of-Thistles
URL: https://www.wikihow.com/Get-Rid-of-Thistles
Sections:
- (No type) Steps  with 9 steps.
   -- Step 1: Systemic herbicide - If you don’t mind killing the surrounding plants, ...
   -- Step 2: Selective weed killer - If your thistles are in a curated garden, opt for ...
   -- Step 3: Competing plants - Perennial grass crops like alfalfa will keep thist...
   -- Step 4: Manual removal - If it’s just an isolated patch of thistles, remove...
   -- Step 5: Mowing - Mowing the thistles before they bloom will curb th...
   -- Step 6: Tilling - Tilling the soil in the fall will weaken the roots...
   -- Step 7: Physical coverage - You can always suffocate the thistles with a flat,...
   -- Step 8: Livestock - Certain animals, like goats and cattle, will gladl...
   -- Step 9: Insects - There are a handful of bugs that will snack on thi...




In [239]:
client_ollama = AsyncClient(
  host='http://192.168.1.5:11434',
  headers={'x-some-header': 'some-value'}
)

async def chat(content):
    message = {'role': 'user', 'content': content}
    response = await client_ollama.chat(model='llama3.2-vision', messages=[message])
    return response['message']['content']


class VLLMClient:
    def __init__(self, system_prompt='A helpful assistant that follow the rule and respond with a structure: Options = []'):
        self.base_url = "http://192.168.1.5:8000/v1"
        self.system = system_prompt
        self.model = self.initialize()

    def initialize(self):
        try:
            response = requests.get(f"{self.base_url}/models")
            data = response.json()
            return data['data'][0]['id']
        except Exception as e:
            raise Exception(f"Failed to initialize the model: {e}, No Connection")

    async def chat(self, prompt: str = '', messages=None, model='', max_tokens: int = 500, options={'temperature':0.4}) -> str:
        if messages is None:
            messages = []

        if messages:
            messages = [{"role": "system", "content": self.system}] + messages
        else:
            messages = [
                {"role": "system", "content": self.system},
                {"role": "user", "content": prompt}
            ]
        if options:
            if 'temperature' in options:
                temperature = options['temperature']

        try:
            async with aiohttp.ClientSession() as session:
                async with session.post(f"{self.base_url}/chat/completions", json={
                    'model': self.model,
                    'messages': messages,
                    'max_tokens': max_tokens,
                    'temperature': temperature
                }) as response:
                    response_data = await response.json()
                    res = {
                        'message': {
                            'role': 'assistant',
                            'content': response_data['choices'][0]['message']['content']
                        }
                    }
                    return res['message']['content']
        except Exception as e:
            return f"An error occurred: {e}"

    async def batch_chat(self, multimessages,options={'temperature':0.4}):
        """Launches multiple chat requests concurrently."""
        tasks = []
        for messages in multimessages:
            tasks.append(asyncio.create_task(self.chat(messages=messages,options=options)))
        results = await asyncio.gather(*tasks)
        return results

client_vllm= VLLMClient() # Initialize the client with a system prompt



In [511]:
# def add_difficulty(difficulty):
#     if not difficulty:
#         return ''
#     elif difficulty == 1:
#         prompt = " The distractors should be about a completely different topic of a step-by-step guide, making them easily distinguishable and distinct from the correct answer." # Adding low difficulty requirement
        
#     elif difficulty == 2:
#         prompt = " The distractors should be about the same topic but really different from the correct answer. Should be easy to choose the correct answer." # Adding low difficulty requirement
#         # Adding medium difficulty requirement
#     elif difficulty == 3:
#         prompt = " The distractors should be somewhat close to the correct answer but conceptually very different, requiring a deep understanding of the topic to differentiate."
#     # Adding high difficulty requirement
#     elif difficulty == 4:
#         prompt = " The distractors should be very close to the correct answer but conceptually different (a different concept), making it hard for someone unfamiliar with the topic to tell them apart."
#     return prompt

def add_difficulty(level):
    if level == 1: # Beginner / Very easy difficulty
        prompt = "The distractors should be about a completely different topic, and represent steps from a different step-by-step guide, making them easily distinguishable from the correct answer."
    elif level == 2: # Novice / Easy difficulty
        prompt = "The distractors should be related to the same topic but significantly different from the correct answer. A basic understanding of the topic is sufficient to choose the right option."
    elif level == 3: # Intermediate difficulty
        prompt = "The distractors should be similar to the correct answer but with some notable differences. A moderate understanding of the topic is required to choose the right option."
    elif level == 4: # Advanced difficulty
        prompt = "The distractors should be subtly different from the correct answer, requiring a deep understanding of the topic to perceive these differences."
    elif level == 5: # Expert / Very hard difficulty
        prompt = "The distractors should be close to the correct answer but conceptually and technically different. Only an expert in this field can easily distinguish between them."
    else:
        return 'Invalid difficulty level. Please enter a number between 0 and 4.'
    return prompt

def find_options(string):
    #string='Options = ["Keep the soil consistently watered, regardless of the weather.", "Wait for the soil to dry out completely before watering again.", "Transplant the baby spider plants to larger pots as soon as possible."]'
    #options_start = string.find("Options =")
    #options=eval(string.split('=')[1].strip())
    #if options:
    #    return options
    #else:
    #    print('Response: ',string)
    #    return None
    options_start = string.find("Options =")
    if options_start != -1:
        start_bracket = string.find("[", options_start)
        end_bracket = string.find("]", options_start)
        if start_bracket != -1 and end_bracket != -1:
            options=''
            try:
                options = list(eval(string[start_bracket + 1:end_bracket]))
            except:
                print(string)
            if options and not options[0].startswith('distractor'):
                    return options
            else:
                print('Distractor',string)
            
    else:
        print('Response: ',string)
        return None

async def generate_distractors(question,correct_answer,difficulty=0):
    prompt='''Given the following question: {question}.\n\nAnd the correct answer: {correct_answer}.\n\nThink of appropriate distractors for a quiz. 
Provide only 3 plausible distractors. '''.format(question=question,correct_answer=correct_answer)+add_difficulty(difficulty)+'''\nAfter considering the question, list the distractors obtained at the end of your response in the form: Options = ["<distractor_1>", "<distractor_2>", "<distractor_3>"], where "distractor_i" are the distractors generated.'''
    #print(prompt)
    #response= await chat(prompt.format(question=question,correct_answer=correct_answer))
    response=await client_vllm.chat(prompt)
    options=find_options(response)
    return options

async def generate_options(question,correct_answer ,difficulty=0):
    options = await  generate_distractors(question,correct_answer,difficulty)
    if not options:
        return None
    options=options[:3]
    options.append(correct_answer)
    random.shuffle(options)
    letters = ['A','B','C','D']
    options = '\n'.join([f'{letters[i]}. {step}' for i, step in enumerate(options)])
    return options

In [512]:
q=results[6]['question']
c_a=results[6]['correct_answer']
#print(q)
#print('\n',c_a)
await generate_distractors(q,c_a,5)

['Secure the press with rope or twine to prevent it from shifting.',
 'Place a weight on top of the press, but not too heavy, as it may damage the specimens.',
 'Leave the press open and allow the specimens to dry naturally in a well-ventilated area.']

In [591]:
question_templates = {
    'missing_step' : 'What step is missing from the "{section_title}" instructions in the "{page_title}" procedure?\n\nSteps:\n{list_of_steps}',
    
    'next_step': 'What is the next step in the "{section_title}" instructions in the "{page_title}" procedure?\n\nSteps:\n{list_of_steps}.', # No option # Option with LLM
    'open_question': 'Write down all the steps from the "{section_title}" instructions in the "{page_title}" procedure.',

    'rearrange': 'Put these steps in the correct order for the "{section_title}" instructions in the "{page_title}" guide.\n\nSteps: {list_of_steps}.', # No LLM
}

async def create_question(page_section, question_type, options_difficulty=1):
    question = question_templates[question_type]
    list_of_steps = [step.title for step in page_section.steps] # Extracting titles of steps
    if len(list_of_steps)<3: # If there are less than three steps, we can't create a question
        return 'Few steps'
    if question_type in ['missing_step']:
        if len(page_section.steps) > 4:
            missing_step_num = random.randint(3,len(page_section.steps)-2)
        else:
            missing_step_num = random.randint(0,len(page_section.steps)-1)
        #correct_answer = page['steps'][missing_step_num]
        correct_answer = list_of_steps[missing_step_num]
        list_of_steps[missing_step_num] = "."*len(correct_answer)
    elif question_type == 'next_step':
        correct_answer = list_of_steps[-1]
        list_of_steps.pop()
    elif question_type == 'rearrange':
        correct_answer = list_of_steps[:]
        correct_answer = '\n'.join([f'{i+1}. {step}' for i, step in enumerate(correct_answer)])
        random.shuffle(list_of_steps)
    elif question_type == 'open_question':
        correct_answer = list_of_steps[:]
        correct_answer = '\n'.join([f'{i+1}. {step}' for i, step in enumerate(correct_answer)])
    list_of_steps = '\n'.join([f'{i+1}. {step}' for i, step in enumerate(list_of_steps)])
    options=None
    if question_type in ['missing_step','next_step']:
        options = await generate_options(question,correct_answer,options_difficulty) # difficulty from 0 to 4
        if not options:
            return None
    else:
        options_difficulty=None
    question=question.format(page_title = page.title.replace('-',' '), section_title = page_section.title , list_of_steps = list_of_steps)
    return {'question' : question, 'options' : options,'correct_answer' : correct_answer, 'options_difficulty': options_difficulty, 'question_type': question_type}


In [514]:
tasks=[]
question_types=["missing_step", "next_step", "open_question", "rearrange"]
sub_pages=list(pages)
for key in tqdm(sub_pages):
    page=pages[key]
    for section in page.sections:
            question_type = random.choice(question_types)
        #for question_type in ["missing_step", "next_step", "open_question", "rearrange"]:
            difficulty=random.randint(1,5)
            if 3<len(section.steps) and question_type=="next_step":
                tasks.append(create_question(section, question_type,difficulty))
            elif 3<len(section.steps)<10: # Check if the section has too much steps to create a question
                tasks.append(create_question(section, question_type,difficulty))
print(len(tasks))
        
        # Process the result here
        # Generate questions of this type for the current section here
        
#await asyncio.gather(*tasks)
            # Generate questions of this type for the current section here


/tmp/ipykernel_29865/487359638.py:1: RuntimeWarning: coroutine 'create_question' was never awaited
  tasks=[]
100%|██████████| 991/991 [00:00<00:00, 133005.51it/s]

2151


In [515]:
results = []
c=0
for task in tqdm(tasks):
    result = await task
    if result is not None:
        results.append(result)
    else:
        print("Result is: ",result)
        c+=1
print(c)

 23%|██▎       | 484/2151 [09:29<32:37,  1.17s/it]  

To determine the next step in the "{section_title}" instructions in the "{page_title}" procedure, we need to analyze the given steps and understand the context.

The steps provided are: {list_of_steps}.

From the context, it seems that the steps are related to creating a stable or decorative arrangement, possibly involving gravel or pebbles. The correct answer, "Adjust where necessary and add pebbles if more are needed," suggests that the goal is to achieve a visually appealing or functional arrangement.

Considering this context, here are three plausible distractors that might require a deep understanding of the topic to distinguish from the correct answer:

Options = ["Adjust where necessary and add additional decorative stones," 
"Make any necessary adjustments and add more pebbles to the surface," 
"Check the arrangement and add pebbles as needed to prevent erosion."]

These distractors are subtly different from the correct answer, requiring an understanding of the context and the 

 53%|█████▎    | 1149/2151 [22:38<13:10,  1.27it/s]

To find a suitable distractor, let's analyze the correct answer: "Check to see if your weed whacker has a “kill switch” that turns it off."

A kill switch is a safety feature on some weed whackers that can be used to quickly turn off the machine in case of an emergency. The correct answer requires the user to be aware of this feature and take the necessary step to ensure their safety.

Here are three plausible distractors:

1. "Check to see if your weed whacker has a "safety lock" that prevents accidental starts." - This distractor is similar to the correct answer, but it's describing a different safety feature (safety lock vs. kill switch) that serves a similar purpose.

2. "Check to see if your weed whacker has a "fuel shut-off" valve that can be used to stop the engine." - This distractor is also describing a safety feature, but it's a different type of feature (fuel shut-off valve vs. kill switch) that serves a similar purpose.

3. "Check to see if your weed whacker has a "maintena

 54%|█████▍    | 1162/2151 [22:59<28:42,  1.74s/it]

...

Response:  To generate plausible distractors, let's consider a different step-by-step guide, such as a guide for a recipe to make a cake.

The correct answer is "Sow the seeds in the soil." This is a step from a gardening or agriculture guide.

Here's a possible correct question:

What step is missing from the "Preparing the Soil" instructions in the "Baking a Delicious Chocolate Cake" procedure?

Steps:
Mix the dry ingredients, Add the eggs one by one, Sift the flour, and Pour the batter into the cake pan.

The correct answer is "Sow the seeds in the soil" is not relevant, but the missing step in the "Preparing the Soil" instructions is "Sow the seeds in the soil" is not in the correct procedure, but the correct answer is "Sift the flour" is not the correct answer, the correct answer is "Sow the seeds in the soil" is not relevant, but the missing step is "Sift the flour" is not the correct answer.

However, the correct answer is not any of the steps in the "Baking a Delicious Chocolat

 73%|███████▎  | 1560/2151 [31:00<17:18,  1.76s/it]

Result is:  None


100%|██████████| 2151/2151 [43:07<00:00,  1.20s/it]

5


In [562]:
def print_pretty(data):
    # Extract question, options, correct answer, and options difficulty from the data.
    question = data['question']
    options = data['options'].split('\n')  # Split the options into individual lines.
    correct_answer = data['correct_answer']
    options_difficulty = data['options_difficulty']

    print("Question:")
    print(question)
    #print(data['options'])
    if data['options']:
        print("\nOptions:")
        for option in options:  # Print each option on a new line.
            print(option)
    print("\nCorrect Answer:")
    print(correct_answer)
    if data['options']:
        print("\nOptions Difficulty Level (1 to 5):",options_difficulty)

In [575]:
num=random.randint(1,len(results))
print_pretty(results[num])

Question:
What step is missing from the "Making a Large Watering Can" instructions in the "Prune Lilacs" procedure?

Steps:
1. Choose a large bottle with a handle and a screw-on lid.
2. Clean out the bottle and remove any labels.
3. Punch holes into the cap with a nail.
4. Consider adding a hole above the handle.
5. ...........................
6. Use the watering can.

Options:
A. Fill the bottle with water.
B. Measure the correct amount of water to fill the bottle.
C. Clean the bottle before filling it with water.
D. Turn off the tap before filling the bottle with water.

Correct Answer:
Fill the bottle with water.

Options Difficulty Level (1 to 5): 2


In [590]:
types={'open_question':0,'rearrange':0, 'next_step':0,'missing_step':0}
difficulties={1:0,2:0,3:0,4:0,5:0,None:0}
for result in results:
    types[result['question_type']]+=1
    difficulties[result['options_difficulty']]+=1
print(types)
print(difficulties)

{'open_question': 477, 'rearrange': 533, 'next_step': 554, 'missing_step': 582}
{1: 229, 2: 233, 3: 241, 4: 215, 5: 218, None: 1010}


In [592]:
results

[{'question': 'What step is missing from the "Propagating Baby Spider Plants in Soil" instructions in the "Prune Lilacs" procedure?\n\nSteps:\n1. Cut the baby spider plants from the tip of the stems.\n2. Fill anursery potwithwell-draining potting soil.\n3. Plant each baby spider plant in its own pot.\n4. ............................................................\n5. Keep the soil moist until you see new growth.',
  'options': 'A. Water the baby spider plants and place them in a sunny windowsill.\nB. Water the baby spider plants and repot them in a larger container.\nC. Water the baby spider plants and fertilize them with a balanced fertilizer.\nD. Water the baby spider plants and set them in indirect light.',
  'correct_answer': 'Water the baby spider plants and set them in indirect light.',
  'options_difficulty': 4,
  'question_type': 'missing_step'},
 {'question': 'Write down all the steps from the "Propagating Baby Spider Plants in Water" instructions in the "Prune Lilacs" proced

In [600]:
import json

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4, ensure_ascii=False)
save_to_json(results,'wikihow.json')

In [ ]:
q_type='open_question'
question= await create_question(page.sections[2],q_type)
print(question['question'])
correct_answer=question['correct_answer']
if q_type in ['something']:
    options = await generate_options(question,correct_answer,difficulty=1) # difficulty from 0 to 4
    print('\nOptions:\n'+options)
print('\nCorrect Answer:\n'+correct_answer)

Write down all the steps for the section "Grafting" in the step-by-step guide "Graft a Fruit Tree".

Correct Answer:
1. Cut the stock.
2. Slice a tongue into the stock branch.
3. Mirror the cut on the scion wood.
4. Match the cut wood together.
5. Tie and cover the graft.
6. Remove the wrapping once new growth develops.
7. Cut the stock branch.
8. Make a cleft into the cut.
9. Trim the scions.
10. Insert the scions into the cleft.
11. Cover the area with grafting compound.
12. Support the growing scions.
13. Gradually cut back the scions.
14. Cut into the bark of the stock.
15. Slice into the stock at an angle.
16. Make sloping cuts into the scion.
17. Join the scion and stock.
18. Nail the scions in place.
19. Coat the wound.
20. Prune as needed.


In [ ]:
#\nOptions:\n'+options+'\nChoose one of the options.
response= await chat(question['question']+'After considering the question, write the correct answer at the end of your response in the form: Correct answer = <correct_answer>')
print(response)

However, I don't see the section on "Collecting Scion Wood" provided. Please provide the section or the relevant information about grafting a fruit tree.

If you'd like, I can still provide general steps for collecting scion wood from common knowledge and experience in grafting fruit trees:

1. **Choose healthy scion branches**: Select one-year-old vigorous branches with at least 2-3 inches (5-7.5 cm) of new growth. These are typically found on the current year's shoots.
2. **Timing is crucial**: Collect scion wood when the tree is dormant, usually in late winter or early spring before budbreak.
3. **Select suitable branches**: Opt for straight, sturdy branches with a diameter between 1/4 inch (6 mm) and 1-1/2 inches (38 mm).
4. **Cut from lower shoots**: Collect scion wood from the lower part of the tree to avoid weak or spindly growth.
5. **Make clean cuts**: Cut the branch at an angle using a sharp, sterile knife or pruning tool, making sure each cut is about 1 inch (2.5 cm) above a